In [1]:
## Gradient Descent is the process of minimizing a function following the 
#  slope or gradient of that function. We can use a technique that 
#  evaluates and updates the coefficients every iteration called 
#  stochastic gradient descent to minimize the error of a model on our 
#  training data.

In [2]:
from random import seed, randrange
from csv import reader
from math import sqrt

In [3]:
# Load a CSV file

def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset


In [4]:
# Convert string column to float

def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())

In [5]:
# Find the min and max values for each column

def dataset_minmax(dataset):
    minmax = list()
    for i in range(len(dataset[0])):
        col_values = [row[i] for row in dataset]
        value_min = min(col_values)
        value_max = max(col_values)
        minmax.append([value_min, value_max])
    return minmax

In [6]:
# Rescale dataset columns to the range 0-1

def normalize_dataset(dataset, minmax):
    for row in dataset:
        for i in range(len(row)):
            row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])

In [7]:
# Split a dataset into k folds

def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for _ in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

In [8]:
# Calculate root mean squared error

def rmse_metric(actual, predicted):
    sum_error = 0.0
    for i in range(len(actual)):
        prediction_error = predicted[i] - actual[i]
        sum_error += (prediction_error ** 2)
    mean_error = sum_error / float(len(actual))
    return sqrt(mean_error)


In [9]:
# Evaluate an algorithm using a cross validation split

def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        rmse = rmse_metric(actual, predicted)
        scores.append(rmse)
    return scores


In [10]:
# Predict Coefficients

def predict(row, coefficients):
    yhat = coefficients[0]
    for i in range(len(row) - 1):
        yhat += coefficients[i + 1] * row[i]
    return yhat

In [11]:
## Gradient Descent

def coefficients_sgd(train, l_rate, n_epoch):
    coef = [0.0 for i in range(len(train[0]))]
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            yhat = predict(row, coef)
            error = yhat - row[-1]
            sum_error += error**2
            coef[0] = coef[0] - l_rate * error
            
            for i in range(len(row) - 1):
                coef[i+1] = coef[i+1] - l_rate * error * row[i]

    return coef

# Modify this later to Change the stochastic gradient descent algorithm to
# accumulate updates across each epoch and only update the coefficients in
# a batch at the end of the epoch.

In [12]:
# def coefficients_sgd(train, l_rate, n_epoch):
#     coef = [0.0 for i in range(len(train[0]))]
#     for epoch in range(n_epoch):
#         sum_error = 0
#         for row in train:
#             yhat = predict(row, coef)
#             error = yhat - row[-1]
#             sum_error += error**2
        
#         coef[0] = coef[0] - l_rate * error
#         for i in range(len(row) - 1):
#             coef[i+1] = coef[i+1] - l_rate * error * row[i]

#     return coef

In [13]:
# Linear Regression Algorithm With Stochastic Gradient Descent

def linear_regression_sgd(train, test, l_rate, n_epoch):
    predictions = list()
    coef = coefficients_sgd(train, l_rate, n_epoch)
    for row in test:
        yhat = predict(row, coef)
        predictions.append(yhat)
    return predictions

In [14]:
# Load Data

filename = 'winequality-white.csv'
dataset = load_csv(filename)

for i in range(len(dataset[0])):
    str_column_to_float(dataset, i)

# Normalize
minmax = dataset_minmax(dataset)
normalize_dataset(dataset, minmax)

# Evaluate Algorithm
n_folds = 10
l_rate = 0.001
n_epoch = 100

scores = evaluate_algorithm(dataset, 
                            linear_regression_sgd, 
                            n_folds, l_rate, n_epoch
                           )

print("Scores: %s" % scores)
print("Mean RMSE: %.3f" % (sum(scores) / float(len(scores))))

Scores: [0.4318709951784056, 0.42194355828810504, 0.42001100130264185, 0.41489738052734504, 0.4249896614511914, 0.420170483912735, 0.4241943606916924, 0.4162484038875378, 0.4164514673343109, 0.40010221378062283]
Mean RMSE: 0.419


In [15]:
# Load Data

filename = 'winequality-red.csv'
dataset = load_csv(filename)

for i in range(len(dataset[0])):
    str_column_to_float(dataset, i)

# Normalize
minmax = dataset_minmax(dataset)
normalize_dataset(dataset, minmax)

# Evaluate Algorithm
n_folds = 15
l_rate = 0.001
n_epoch = 100

scores = evaluate_algorithm(dataset, 
                            linear_regression_sgd, 
                            n_folds, l_rate, n_epoch
                           )

print("Scores: %s" % scores)
print("Mean RMSE: %.3f" % (sum(scores) / float(len(scores))))

Scores: [0.33262367822435085, 0.337809041849811, 0.34236855432005864, 0.32094959227669545, 0.32033134245694134, 0.3436898189998329, 0.31324020670977804, 0.3184755546765138, 0.3093821277804809, 0.35746993405645944]
Mean RMSE: 0.330
